In [2]:
import pandas as pd

fire_combined_df_augmented = pd.read_csv('fire_events_with_weather_metrics_augmented.csv')


In [9]:
print(fire_combined_df_augmented.columns)

Index(['id', 'geometry', 'ig_date', 'ig_day', 'ig_month', 'ig_year',
       'last_date', 'event_dur', 'tot_pix', 'tot_ar_km2',
       ...
       'fwi_proxy_day14', 'fwi_proxy_7d', 'wind_temp_day1', 'wind_temp_day3',
       'wind_temp_day5', 'wind_temp_day7', 'wind_temp_trend',
       'wind_dir_stability_7d', 'temp_trend_7d', 'fwi_high_days_14d'],
      dtype='object', length=335)


In [3]:
numerical_features = fire_combined_df_augmented.columns[35:].tolist()
numerical_features

['temperature_2m_max_mean',
 'temperature_2m_max_median',
 'temperature_2m_max_min',
 'temperature_2m_max_max',
 'temperature_2m_max_std',
 'temperature_2m_max_range',
 'temperature_2m_max_q25',
 'temperature_2m_max_q75',
 'temperature_2m_max_iqr',
 'temperature_2m_max_last3_mean',
 'temperature_2m_max_last7_mean',
 'temperature_2m_min_mean',
 'temperature_2m_min_median',
 'temperature_2m_min_min',
 'temperature_2m_min_max',
 'temperature_2m_min_std',
 'temperature_2m_min_range',
 'temperature_2m_min_q25',
 'temperature_2m_min_q75',
 'temperature_2m_min_iqr',
 'temperature_2m_min_last3_mean',
 'temperature_2m_min_last7_mean',
 'apparent_temperature_max_mean',
 'apparent_temperature_max_median',
 'apparent_temperature_max_min',
 'apparent_temperature_max_max',
 'apparent_temperature_max_std',
 'apparent_temperature_max_range',
 'apparent_temperature_max_q25',
 'apparent_temperature_max_q75',
 'apparent_temperature_max_iqr',
 'apparent_temperature_max_last3_mean',
 'apparent_temperature_

Wind-speed interaction

In [11]:
import numpy as np

fire_combined_df_augmented['wind_speed_direction'] = \
    fire_combined_df_augmented['wind_direction_10m_dominant_mean'] * \
    fire_combined_df_augmented['wind_speed_10m_max_mean']

fire_combined_df_augmented['wind_aspect_alignment_day1'] = \
    np.cos(np.radians(fire_combined_df_augmented['wind_direction_10m_dominant_day1'] - fire_combined_df_augmented['slope_aspect']))

fire_combined_df_augmented['upslope_wind_speed_day1'] = \
    fire_combined_df_augmented['wind_speed_10m_max_day1'] * \
    fire_combined_df_augmented['wind_aspect_alignment_day1']


Fire Weather Index Proxy

In [17]:
fire_combined_df_augmented['dry_heat_index_day1'] = \
    fire_combined_df_augmented['temperature_2m_max_day1'] - \
    fire_combined_df_augmented['precipitation_sum_day1']

# drought-stress index
fire_combined_df_augmented['drought_stress_index'] = \
    fire_combined_df_augmented['dryness_7d'] * \
    (1 - fire_combined_df_augmented['fuel_moisture_7d'])

# Normalize different areas
fire_combined_df_augmented['wind_speed_to_gust_ratio_day1'] = \
    fire_combined_df_augmented['wind_speed_10m_max_day1'] / \
    (fire_combined_df_augmented['wind_gusts_10m_max_day1'] + 1e-3)



In [18]:
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

categorical_features = ['eco_name', 'lc_name']

# --- Initial split (80/20) ---
X_full = fire_combined_df_augmented[numerical_features + categorical_features + ['wind_speed_to_gust_ratio_day1', 'drought_stress_index', 'dry_heat_index_day1']].copy()
y_full = fire_combined_df_augmented['fsr_km2_dy'].copy()
valid_idx = ~y_full.isna() & X_full[numerical_features].notna().all(axis=1)

In [19]:
X_full = X_full.loc[valid_idx]
y_full = y_full.loc[valid_idx]

X_train_full, X_test, y_train_full, y_test = train_test_split(
    X_full, y_full, test_size=0.2, random_state=42
)

# --- Sample subset for fast MI + correlation ---
sample_size = min(5000, len(X_train_full))  # Cap for speed
X_train_sample = X_train_full[numerical_features].sample(n=sample_size, random_state=42)
y_train_sample = y_train_full.loc[X_train_sample.index]

# --- Compute mutual information ---
mi = mutual_info_regression(X_train_sample, y_train_sample, random_state=42)
mi_series = pd.Series(mi, index=numerical_features).sort_values(ascending=False)
top_k = 60  # Adjust as needed
selected_by_mi = mi_series.head(top_k).index.tolist()

# --- Correlation pruning ---
X_top = X_train_sample[selected_by_mi]
corr_matrix = X_top.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [col for col in upper.columns if any(upper[col] > 0.95)]
final_selected_features = list(set(selected_by_mi) - set(to_drop))

print(f"\nSelected {len(final_selected_features)} features for DNN:")
print(final_selected_features)


Selected 40 features for DNN:
['precipitation_sum_trend', 'apparent_temperature_max_iqr', 'wind_direction_10m_dominant_q25', 'precipitation_hours_day3', 'apparent_temperature_min_max', 'precipitation_hours_day5', 'wind_east_velocity', 'shortwave_radiation_sum_day5', 'apparent_temperature_max_day7', 'temperature_2m_max_last3_mean', 'wind_temp_day7', 'precipitation_sum_std', 'sunshine_duration_day3', 'daylight_duration_last3_mean', 'daylight_duration_trend', 'precipitation_hours_last7_mean', 'wind_speed_10m_max_max', 'wind_speed_10m_max_trend', 'et0_fao_evapotranspiration_q25', 'temperature_2m_max_min', 'snowfall_sum_min', 'wind_eastward', 'apparent_temperature_max_day5', 'wind_direction_10m_dominant_q75', 'shortwave_radiation_sum_day3', 'apparent_temperature_min_std', 'sunshine_duration_last3_mean', 'rain_sum_median', 'wind_gusts_10m_max_std', 'sunshine_duration_last7_mean', 'precipitation_sum_day2', 'precipitation_hours_sum', 'sunshine_duration_min', 'temperature_2m_max_iqr', 'wind_sp

In [23]:
print('wind_speed_to_gust_ratio_day1' in final_selected_features)

False


In [24]:
final_selected_features += ['wind_speed_to_gust_ratio_day1', 'drought_stress_index', 'dry_heat_index_day1']

In [25]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l1_l2
import scipy.stats as stats

categorical_features = ['eco_name', 'lc_name']
target = 'fsr_km2_dy'

# Subset and clean
X = fire_combined_df_augmented[final_selected_features + categorical_features].copy()
y = fire_combined_df_augmented[target].copy()
valid_idx = ~y.isna() & X.notna().all(axis=1)
X = X.loc[valid_idx]
y = y.loc[valid_idx]

# Split without log transform
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)

print(len(X_train_full))
print(len(X_train))
print(len(X_val))
print(len(X_test))

97064
77651
19413
24266


In [26]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l1_l2
import scipy.stats as stats

categorical_features = ['eco_name', 'lc_name']
target = 'fsr_km2_dy'

# Subset and clean
X = fire_combined_df_augmented[final_selected_features + categorical_features].copy()
y = fire_combined_df_augmented[target].copy()
valid_idx = ~y.isna() & X.notna().all(axis=1)
X = X.loc[valid_idx]
y = y.loc[valid_idx]

# Remove outliers in target variable
# z_scores = np.abs(stats.zscore(y))
# outlier_mask = z_scores < 3
# X = X[outlier_mask]
# y = y[outlier_mask]

# Split without log transform
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)

print(len(X_train_full))
print(len(X_train))
print(len(X_val))
print(len(X_test))

# Preprocess
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), final_selected_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])
X_train_proc = preprocessor.fit_transform(X_train)
X_val_proc = preprocessor.transform(X_val)
X_test_proc = preprocessor.transform(X_test)

# More complex DNN definition
def build_dnn(input_dim):
    model = Sequential([
        Dense(512, kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4), input_dim=input_dim),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.3),

        Dense(256, kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.3),

        Dense(128, kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.2),

        Dense(64, kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.1),

        Dense(32),
        LeakyReLU(alpha=0.1),

        Dense(1)
    ])
    # Using Huber loss instead of MSE to be more robust to outliers
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                  loss=tf.keras.losses.Huber(delta=1.0))
    return model

# R² Callback
class ValR2Callback(tf.keras.callbacks.Callback):
    def __init__(self, X_val, y_val):
        self.X_val = X_val
        self.y_val = y_val

    def on_epoch_end(self, epoch, logs=None):
        y_pred = self.model.predict(self.X_val, verbose=0).flatten()
        r2 = r2_score(self.y_val, y_pred)
        print(f" val_R² = {r2:.4f}")

# Train with more patience and better scheduling
model = build_dnn(X_train_proc.shape[1])
early_stop = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
lr_sched = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-7, verbose=1)
val_r2_cb = ValR2Callback(X_val_proc, y_val)

# Using a larger batch size (256 instead of 128)
history = model.fit(
    X_train_proc, y_train,
    validation_data=(X_val_proc, y_val),
    epochs=200,
    batch_size=256,  # Increased batch size for better stability and faster training
    callbacks=[early_stop, lr_sched, val_r2_cb],
    verbose=1
)

# Evaluate
y_pred = model.predict(X_test_proc).flatten()
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

print("\nFinal DNN Performance on Test Set:")
print(f"R²     = {r2:.4f}")
print(f"RMSE   = {rmse:.4f}")
print(f"MAE    = {mae:.4f}")

r2, rmse, mae

97064
77651
19413
24266


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/200
302/304 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.5590 val_R² = -0.0355
304/304 ━━━━━━━━━━━━━━━━━━━━ 17s 36ms/step - loss: 0.5583 - val_loss: 0.3906 - learning_rate: 1.0000e-04
Epoch 2/200
303/304 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.3868 val_R² = -0.0141
304/304 ━━━━━━━━━━━━━━━━━━━━ 8s 27ms/step - loss: 0.3868 - val_loss: 0.3607 - learning_rate: 1.0000e-04
Epoch 3/200
301/304 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.3610 val_R² = -0.0076
304/304 ━━━━━━━━━━━━━━━━━━━━ 11s 30ms/step - loss: 0.3610 - val_loss: 0.3494 - learning_rate: 1.0000e-04
Epoch 4/200
302/304 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.3420 val_R² = -0.0047
304/304 ━━━━━━━━━━━━━━━━━━━━ 10s 30ms/step - loss: 0.3421 - val_loss: 0.3419 - learning_rate: 1.0000e-04
Epoch 5/200
303/304 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.3464 val_R² = -0.0044
304/304 ━━━━━━━━━━━━━━━━━━━━ 10s 31ms/step - loss: 0.3463 - val_loss: 0.3355 - learning_rate: 1.0000e-04
Epoch 6/200
302/304 ━━━━━━━━━━━━━━━━━━━━ 0s 20

(0.018216122240937604, np.float64(1.6621725723601868), 0.29269227506369305)